<a href="https://colab.research.google.com/github/beaumontslim/misc/blob/main/mlflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mlflow

     |████████████████████████████████| 14.2MB 234kB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
     |████████████████████████████████| 153kB 39.9MB/s 
     |████████████████████████████████| 163kB 40.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=29bad4ed82872efed776d79fcfb83235b4318550bad0ce51e3b5cc5ab006bb28
  Stored in directory: /root/.cache/pip/wheels/84/07/f7/12f7370ca47a66030c2edeedcc23dec26ea0ac22dcb4c4a0f3
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.1-cp37-none-any.whl size=17159 sha256=aaa4c2089156de83b251bf8e2bd78c08fbd49d9f4b71be8f427855a45aeadaae
  Stored in directory: /root/.cache/pip/wheels/b4/1f

In [3]:
!pip install pyngrok

     |████████████████████████████████| 747kB 6.5MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=3e89c5d4b53884f1816bd739eb0a4b507d8b3eedc5e39fa75b38e7cfbb32c9cb
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [4]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [5]:
from pyngrok import ngrok

In [6]:
ngrok.kill()
ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print("MLflow UI ", ngrok_tunnel.public_url)

MLflow UI  https://7f6fb7d2b83d.ngrok.io


In [9]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

if __name__ == '__main__':

    mlflow.set_experiment(experiment_name='mlflow demo bro')
    
    training_data = pd.read_csv('https://raw.github.com/futurexskill/ml-model-deployment/main/storepurchasedata.csv')
    
    print('loaded training data')
    
    training_data.head()
    
    training_data.describe()
    mlflow.log_param('training percentage',70)
    
    #split IVs and DV 
    
    x = training_data.iloc[:,:-1].values
    y = training_data.iloc[:,-1].values
    
    #train test split process
    
    from sklearn.model_selection import train_test_split
    
    x_train, x_test, y_train, y_test, = train_test_split(x,y, test_size = 0.3, random_state = None)
     
    #scale the data
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    print('completed feature scaling')
    
    ## build classification model, use KNN in this example, N being 5 for 5 nearest neighbors, metric being Minkoski for
    ## Euclidean distance
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski')
    
    #model training
    classifier.fit(x_train, y_train)
    
    print('model trained!')
    
    #use model to predict and find probabilities
    
    y_pred = classifier.predict(x_test)
    y_prob = classifier.predict_proba(x_test)[:,1]
    
    #build confusion matrix
    
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    #get accuracy. in the last test, we got 90. for some reason the first time around is was crap?
    
    from sklearn.metrics import accuracy_score
    
    model_accuracy = accuracy_score(y_test,y_pred)
    
    print(model_accuracy)
    
    mlflow.log_metric("accuracy", model_accuracy)
    mlflow.sklearn.log_model(classifier, 'model')
    
    #get classfication report
    from sklearn.metrics import classification_report
    print(classification_report((y_test), y_pred))
    
    #get predictions for specific ages/salaries. 40/50k gets a .6, thus a green light for a purchase
    
    new_prediction = classifier.predict(sc.transform((np.array([[40,50000]]))))
    new_proba = classifier.predict_proba(sc.transform((np.array([[40,50000]]))))
    
    #pickle the classifier and the scaler
    
    import pickle
    
    model_file = 'classifier.pickle'
    pickle.dump(classifier, open(model_file, 'wb'))
    scaler_file = 'sc.pickle'
    pickle.dump(sc,open(scaler_file, 'wb'))

loaded training data
completed feature scaling
model trained!
0.8333333333333334
              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.83      0.83      0.83         6

    accuracy                           0.83        12
   macro avg       0.83      0.83      0.83        12
weighted avg       0.83      0.83      0.83        12

